# PAQUETES

In [ ]:
#Importamos las liberías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as pltimg
import seaborn as sns
%matplotlib inline

from pandas import read_csv
import matplotlib.pyplot as plt
import matplotlib.image as pltimg
#import pydotplus
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix , ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.model_selection import GridSearchCV 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.utils import class_weight

#import warnings
#warnings.filterwarnings("ignore")

# PREPARACIÓN DE DATOS

In [ ]:
#Cargamos el csv
df=pd.read_csv("cleveland_cardiaco.csv",sep=",")
#Añadimos los nombres de las columnas
headers = ["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num"]
df.columns = headers
df

In [ ]:
#Vemos los valores que tiene cada columna y cuantas veces aparece cada uno de ellos
for i in df.columns:
    print(i,df[i].value_counts())

In [ ]:
#Información del dataset
df.info()

In [ ]:
# Borramos las filas en las que aparezca el valor "?"
df=df[df!="?"].dropna()

In [ ]:
# Información del dataset
df.info()

In [ ]:
df['age'] = df['age'].astype(int)
df['sex'] = df['sex'].astype(int)
df['cp'] = df['cp'].astype(int)
df['trestbps'] = df['trestbps'].astype(int)
df['chol'] = df['chol'].astype(int)
df['fbs'] = df['fbs'].astype(int)
df['restecg'] = df['restecg'].astype(int)
df['thalach'] = df['thalach'].astype(int)
df['exang'] = df['exang'].astype(int)
df['slope'] = df['slope'].astype(int)
df['oldpeak'] = df['oldpeak'].astype(float)
df['thal'] = df['thal'].astype(float)
df['ca'] = df['ca'].astype(float)
df['thal'] = df['thal'].astype(float)
df['ca'] = df['ca'].astype(int)
df['thal'] = df['thal'].astype(int)
df['num'] = df['num'].astype(int)

In [ ]:
# Guardamos el dataset como csv
df.to_csv("cleveland_cardiaco_py.csv", index=False)

In [ ]:
# Calcula la frecuencia de los valores de clase en todo el DataFrame
frecuencia_clase = df['num'].value_counts()

# Crea un gráfico de barras para visualizar la frecuencia de los valores de clase
plt.bar(frecuencia_clase.index, frecuencia_clase.values)
plt.xlabel('Diagnóstico')
plt.ylabel('Frecuencia')
plt.show()


# PREPROCESAMIENTO

In [ ]:
# Dividimos entrada y salida
x = df.iloc[:, :-1]  # Características (atributos)
y = df.iloc[:, -1]  # Etiquetas (salida)
# Dividimos en datos de entrenamiento y de validación
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, shuffle=True)
x_train.columns = ["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal"]
x_test.columns = ["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal"]

#Tamaños
x.shape, y.shape

x_train.shape, y_train.shape

x_test.shape, y_test.shape

# Calculamos el máximo y mínimo
maxs = np.max(x_train, axis=0)
mins = np.min(x_train, axis=0)

ranges = maxs - mins

# Normalizamos las variables
x_train = (x_train - mins) / ranges
x_test = (x_test - mins) / ranges

# Calcula las frecuencias de clase en y_train
class_frequencies = y_train.value_counts()

# Calcula los pesos para cada clase  
#class_weights = {cls: 1.0 / freq for cls, freq in class_frequencies.items()}
#class_weights_int = {int(cls[0]): weight for cls, weight in class_weights.items()}

# Calcula los pesos de clase
class_weights = class_weight.compute_class_weight('balanced', classes=y.unique(), y=y)

# Crea un diccionario con los pesos de clase
class_weights_int = {cls: weight for cls, weight in zip(y.unique(), class_weights)}


# MODELOS

## Árbol de decisión

In [ ]:
# modelo genérico
DT = DecisionTreeClassifier(class_weight=class_weights_int)
# entrenamos el modelo
DT.fit(x_train, y_train)
# hacer predicciones
pred_DT = DT.predict(x_test)
# informe
print(classification_report(y_test,pred_DT))
# cross value
kfold = StratifiedKFold(10)
score = cross_val_score(
    DT, x_train, y_train, cv=kfold, scoring="accuracy"
)
DT_prec=score.mean()
print("cross_value: ", DT_prec)
# matriz de confusión
mat_conf = confusion_matrix(y_test, pred_DT)
ConfusionMatrixDisplay(mat_conf).plot()
plt.show()

In [ ]:
# ver opciones con otros parametros
DT_combinaciones={}
# parametros
splitter_ = ["best", "random"]
criterion_ = ["gini", "entropy"]
max_features_ = [None,"sqrt", "log2"]
porc_ = [0.2, 0.3]
# combinaciones
for c in criterion_:
    for m in max_features_:
        for s in splitter_:
            for p in [1, 2]:
                vector_parametros=[c,m,s,porc_[p-1]]
                # dividimos las instancias
                x_train, x_test, y_train, y_test = train_test_split(
                    x, y, test_size=porc_[p - 1], random_state=42, shuffle=True
                )
                x_train.columns = ["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal"]
                x_test.columns = ["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal"]

                # modelo
                DT = DecisionTreeClassifier(criterion=c, max_features=m, splitter=s, class_weight=class_weights_int)
                # entrenamos el modelo
                DT.fit(x_train, y_train)
                # hacer predicciones
                predicciones_DT = DT.predict(x_test)
                # calculamos la exactitud del modelo
                acc_1 = accuracy_score(y_test, predicciones_DT)
                # cross value
                kfold = StratifiedKFold(10)
                score = cross_val_score(DT, x_train, y_train, cv=kfold, scoring="accuracy")
                print(
                    c,
                    ",",
                    m,
                    "y",
                    s,
                    "(",
                    porc_[p - 1],
                    ")",
                    "-> accuracy_score: ",
                    acc_1,
                    "//",
                    "cross_value: ",
                    score.mean()
                )
                # añadimos al diccionario esta combinación
                DT_combinaciones[score.mean()]=vector_parametros
                
# calculamos la precisión máxima
DT_max_prec = max(DT_combinaciones)
# miramos con que parámetros se consigue esta precisión
DT_mejor_vector_parametros = DT_combinaciones[DT_max_prec]
print("mejores resultados con los parametros -> ",DT_mejor_vector_parametros)

In [ ]:
# mejor modelo (utilizando el vector conseguido anteriormente)
DT_mejor = DecisionTreeClassifier(criterion=DT_mejor_vector_parametros[0], max_features=DT_mejor_vector_parametros[1], splitter=DT_mejor_vector_parametros[2], class_weight=class_weights_int)
# dividimos las instancias 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=DT_mejor_vector_parametros[3], random_state=42, shuffle=True)

# entrenamos el modelo
DT_mejor.fit(x_train, y_train)
# hacer predicciones
pred_DT_mejor = DT_mejor.predict(x_test)
# informe
print(classification_report(y_test,pred_DT_mejor,zero_division=1))
# cross value
kfold = StratifiedKFold(10)
score = cross_val_score(
    DT_mejor, x_train, y_train, cv=kfold, scoring="accuracy"
)
DT_mejor_prec=score.mean()
print("cross_value: ", DT_mejor_prec)
# matriz de confusión
mat_conf = confusion_matrix(y_test, pred_DT_mejor)
ConfusionMatrixDisplay(mat_conf).plot()
plt.show()

## Random forest

In [ ]:
# modelo genérico
y_ravel= np.ravel(y)
RF = RandomForestClassifier(class_weight=class_weights_int)
# entrenamos el modelo
y_train=y_train.values.ravel()
RF.fit(x_train, y_train)
# hacer predicciones
pred_RF = RF.predict(x_test)
# informe
print(classification_report(y_test,pred_RF,zero_division=0))
# cross value
kfold = StratifiedKFold(10)
score = cross_val_score(
    RF, x_train, y_train, cv=kfold, scoring="accuracy"
)
RF_prec=score.mean()
print("cross_value: ", RF_prec)
# matriz de confusión
mat_conf = confusion_matrix(y_test, pred_RF)
ConfusionMatrixDisplay(mat_conf).plot()
plt.show()

In [ ]:
# ver opciones con otros parámetros
RF_combinaciones={}
# parametros
criterion_ = ["gini", "entropy"]
max_features_ = [None,"sqrt", "log2"]
porc_ = [0.2, 0.3]
# combinaciones
for c in criterion_:
    for m in max_features_:
        for p in [1, 2]:
            vector_parametros=[c,m,porc_[p-1]]
            # dividimos las instancias
            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=porc_[p - 1], random_state=42, shuffle=True)
            y_train=y_train.values.ravel()
            # modelo
            RF = RandomForestClassifier(criterion=c, max_features=m, class_weight=class_weights_int)
            # entrenamos el modelo
            RF.fit(x_train, y_train)
            # hacer predicciones
            predicciones_RF = RF.predict(x_test)
            # calculamos la exactitud del modelo
            acc_1 = accuracy_score(y_test, predicciones_RF)
            # cross value
            kfold = StratifiedKFold(10)
            score = cross_val_score(RF, x_train, y_train, cv=kfold, scoring="accuracy")
            print(
                c,
                "y",
                m,
                "(",
                porc_[p - 1],
                ")",
                "-> accuracy_score: ",
                acc_1,
                "//",
                "cross_value: ",
                score.mean()
            )
            # añadimos al diccionario esta combinación
            RF_combinaciones[score.mean()]=vector_parametros
                
# calculamos la precisión máxima
RF_max_prec = max(RF_combinaciones)
# miramos con que parámetros se consigue esta precisión
RF_mejor_vector_parametros = RF_combinaciones[RF_max_prec]
print("mejores resultados con los parametros -> ",RF_mejor_vector_parametros)

In [ ]:
# mejor modelo (utilizando el vector conseguido anteriormente)
RF_mejor = RandomForestClassifier(criterion=RF_mejor_vector_parametros[0], max_features=RF_mejor_vector_parametros[1], class_weight=class_weights_int)
# dividimos las instancias 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=RF_mejor_vector_parametros[2], random_state=42, shuffle=True)
y_train=y_train.values.ravel()
# entrenamos el modelo
RF_mejor.fit(x_train, y_train)
# hacer predicciones
pred_RF_mejor = RF_mejor.predict(x_test)
# informe
print(classification_report(y_test,pred_RF_mejor,zero_division=1))
# cross value
kfold = StratifiedKFold(10)
score = cross_val_score(
    RF_mejor, x_train, y_train, cv=kfold, scoring="accuracy"
)
RF_mejor_prec=score.mean()
print("cross_value: ", RF_mejor_prec)
# matriz de confusión
mat_conf = confusion_matrix(y_test, pred_RF_mejor)
ConfusionMatrixDisplay(mat_conf).plot()
plt.show()

## Redes neuronales

In [ ]:
optim = ["adam", "SGD", "RMSprop", "adagrad", "adamax"]
RN_combinaciones = {}

# Separa los atributos y las etiquetas de diagnóstico
x = df.iloc[:, :-1].values
y = df.iloc[:, -1].values 

# Divide los datos en conjuntos de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Escala los atributos para normalizarlos
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# ESTRUCTURA 1
RNmodel = Sequential()

# Agrega capas densas (totalmente conectadas) al modelo
RNmodel.add(Dense(64, activation='relu', input_shape=(13,)))  # Capa oculta 1
RNmodel.add(Dense(64, activation='relu'))  # Capa oculta 2
RNmodel.add(Dense(5, activation='softmax'))  # Capa de salida (5 valores posibles)

for i in optim:
    # Compila el modelo
    RNmodel.compile(optimizer=i, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Entrena el modelo
    RNmodel.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

    y_pred_prob = RNmodel.predict(x_test)
    y_pred = np.argmax(y_pred_prob, axis=1)

    # Calcular precisión
    precision = accuracy_score(y_test, y_pred)
    print("Precisión:", precision)

    # Calcular matriz de confusión
    matriz_confusion = confusion_matrix(y_test, y_pred)
    print("Matriz de confusión:")
    ConfusionMatrixDisplay(matriz_confusion).plot()
    plt.show()

    RN_combinaciones[precision] = [1, i]

# ESTRUCTURA 2
RNmodel = Sequential()

# Agrega capas densas (totalmente conectadas) al modelo
RNmodel.add(Dense(128, activation='relu', input_shape=(13,)))  # Capa oculta 1
RNmodel.add(Dense(64, activation='relu'))  # Capa oculta 2
RNmodel.add(Dense(32, activation='relu'))  # Capa oculta 3
RNmodel.add(Dense(5, activation='softmax'))  # Capa de salida (5 valores posibles)

for i in optim:
    # Compila el modelo
    RNmodel.compile(optimizer=i, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Entrena el modelo
    RNmodel.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

    y_pred_prob = RNmodel.predict(x_test)
    y_pred = np.argmax(y_pred_prob, axis=1)

    # Calcular precisión
    precision = accuracy_score(y_test, y_pred)
    print("Precisión:", precision)

    # Calcular matriz de confusión
    matriz_confusion = confusion_matrix(y_test, y_pred)
    print("Matriz de confusión:")
    ConfusionMatrixDisplay(matriz_confusion).plot()
    plt.show()

    RN_combinaciones[precision] = [2, i]

# ESTRUCTURA 3
RNmodel = Sequential()

# Agrega capas densas (totalmente conectadas) al modelo
RNmodel.add(Dense(64, activation='relu', input_shape=(13,)))  # Capa oculta 1
RNmodel.add(Dense(128, activation='relu'))  # Capa oculta 2
RNmodel.add(Dense(64, activation='relu'))  # Capa oculta 3
RNmodel.add(Dense(32, activation='relu'))  # Capa oculta 4
RNmodel.add(Dense(5, activation='softmax'))  # Capa de salida (5 valores posibles)

for i in optim:
    # Compila el modelo
    RNmodel.compile(optimizer=i, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Entrena el modelo
    RNmodel.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

    y_pred_prob = RNmodel.predict(x_test)
    y_pred = np.argmax(y_pred_prob, axis=1)

    # Calcular precisión
    precision = accuracy_score(y_test, y_pred)
    print("Precisión:", precision)

    # Calcular matriz de confusión
    matriz_confusion = confusion_matrix(y_test, y_pred)
    print("Matriz de confusión:")
    ConfusionMatrixDisplay(matriz_confusion).plot()
    plt.show()

    RN_combinaciones[precision] = [3, i]

print(RN_combinaciones)
# calculamos la precisión máxima
RN_max_prec = max(RN_combinaciones)
# miramos con qué parámetros se consigue esta precisión
RN_mejor_vector_parametros = RN_combinaciones[RN_max_prec]
print("Mejores resultados con los parámetros:", RN_mejor_vector_parametros)


In [ ]:
# Mejor modelo

RNmodel_mejor = Sequential()

if RN_mejor_vector_parametros[0]==1:
    RNmodel_mejor.add(Dense(64, activation='relu', input_shape=(13,)))  # Capa oculta 1
    RNmodel_mejor.add(Dense(64, activation='relu'))  # Capa oculta 2
    RNmodel.add(Dense(5, activation='softmax'))  # Capa de salida (5 valores posibles)
elif RN_mejor_vector_parametros[0]==2:
    RNmodel_mejor.add(Dense(128, activation='relu', input_shape=(13,)))  # Capa oculta 1
    RNmodel_mejor.add(Dense(64, activation='relu'))  # Capa oculta 2
    RNmodel_mejor.add(Dense(32, activation='relu'))  # Capa oculta 3
    RNmodel_mejor.add(Dense(5, activation='softmax'))  # Capa de salida (5 valores posibles)
elif RN_mejor_vector_parametros[0]==3:
    RNmodel_mejor.add(Dense(64, activation='relu', input_shape=(13,)))  # Capa oculta 1
    RNmodel_mejor.add(Dense(128, activation='relu'))  # Capa oculta 2
    RNmodel_mejor.add(Dense(64, activation='relu'))  # Capa oculta 3
    RNmodel_mejor.add(Dense(32, activation='relu'))  # Capa oculta 4
    RNmodel_mejor.add(Dense(5, activation='softmax'))  # Capa de salida (5 valores posibles)
# Compila el modelo
RNmodel_mejor.compile(optimizer=RN_mejor_vector_parametros[1], loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entrena el modelo
RNmodel_mejor.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

y_pred_prob = RNmodel_mejor.predict(x_test)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calcular precisión
RN_mejor_prec = accuracy_score(y_test, y_pred)
print("Precisión:", RN_mejor_prec)

# Calcular matriz de confusión
matriz_confusion = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:")
ConfusionMatrixDisplay(matriz_confusion).plot()
plt.show()

## KNN

In [ ]:
avg_score=[]
max_score=0
best_k=0
for k in range(2,30):
    knn_model=KNeighborsClassifier(n_jobs=-1,n_neighbors=k)
    score=cross_val_score(knn_model,x_train,y_train,cv=5,scoring='accuracy')
    if score.mean()>max_score:
        max_score=score.mean()
        best_k=k
    avg_score.append(score.mean())

print([best_k,max(avg_score)])

max_index = avg_score.index(max(avg_score))
max_value = max(avg_score)

plt.figure(figsize=(15, 8))
plt.plot(range(2, 30), avg_score, color='blue', linewidth=2)  
plt.scatter(max_index+2, max_value, color='darkblue', marker='o', s=200)  
plt.xlabel("k neighbors")
plt.ylabel("accuracy")

plt.show()

In [ ]:
# modelo
KNN=KNeighborsClassifier(n_neighbors=best_k)
# entrenamos el modelo
KNN.fit(x_train,y_train)
# hacer predicciones
pred_KNN = KNN.predict(x_test)
# informe
print(classification_report(y_test,pred_KNN,zero_division=0))
# cross value
kfold = StratifiedKFold(10)
score = cross_val_score(
    KNN, x_train, y_train, cv=kfold, scoring="accuracy"
)
KNN_prec=score.mean()
print("cross_value: ", KNN_prec)
# matriz de confusión
mat_conf = confusion_matrix(y_test, pred_KNN)
ConfusionMatrixDisplay(mat_conf).plot()
plt.show()

# DIAGNOSIS

In [ ]:
nombre_v=str(input("Ahora introduciras los datos médicos del paciente con nombre o identificador: "))

age_v=int(input("edad en años: "))
sex_v=int(input("sexo (0=mujer)(1=hombre): "))
cp_v=int(input("tipo de angina (1=típico)(2=atípico)(3=no es angina)(4=asintomático): "))
trestbps_v=int(input("presión sanguínea entre latidos en mm Hg: "))
chol_v=int(input("colesterol en sangre en mg/dL: "))
fbs_v=int(input("glucosa en sangre en ayunas (0=menos de 120 mg/dL)(1=más de 120 mg/dL): "))
restecg_v=int(input("reultado electrocardiográfico en reposo (0=normal)(1=anormal en la onda ST-T)(2=hipertrofia ventricular izq): "))
thalach_v=int(input("frecuencia carduaca máxima durante el ejercicio: "))
exang_v=int(input("angina durante el ejercicio (0=no)(1=sí): "))
oldpeak_v=float(input("diferencia de altura del segmento ST: "))
slope_v=int(input("pendiente del segmento ST durante el ejercicio máximo (1=ascendente)(2=plano)(3=descendente): "))
ca_v=int(input("número de vasos principales colorados durante la fluoroscopia (de 0 a 3): "))
thal_v=int(input("diagnóstico de la enfermedad cardiaca (3=sin defectos)(6=defecto fijo)(7=defecto reversible): "))

In [ ]:
pac=[age_v, sex_v, cp_v, trestbps_v,chol_v,fbs_v,restecg_v,thalach_v, exang_v, oldpeak_v, slope_v, ca_v, thal_v]
#pac=[65,1,4,162,287,0,2,107,1,1.5,2,3,3]
#nombre_v="Aitor"

#Crear los DataFrames que necesitaremos
df_diagnostico = pd.DataFrame(columns=['Método', 'Características', 'Precisión', 'Diagnóstico'])

df_datos=pd.DataFrame({'medida':['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal'],
                     'valor':pac})

df_nombre=pd.DataFrame({'nombre':[nombre_v]})

# Predecir diagnóstico con diferentes métodos
DT_predict=int(DT_mejor.predict([pac]))
RF_predict=int(RF_mejor.predict([pac]))
pac_scaled = scaler.transform([pac])  # Escala el vector pac
y_pred_prob = RNmodel_mejor.predict(pac_scaled)
RN_predict = np.argmax(y_pred_prob, axis=1)
KNN_predict=int(KNN.predict([pac]))
    
df_diagnostico = df_diagnostico.append({'Método': 'Árbol de decisión', 'Características': DT_mejor_vector_parametros, 'Precisión': DT_mejor_prec, 'Diagnóstico': DT_predict}, ignore_index=True)
df_diagnostico = df_diagnostico.append({'Método': 'Random Forest', 'Características': RF_mejor_vector_parametros, 'Precisión': RF_mejor_prec, 'Diagnóstico': RF_predict}, ignore_index=True)
df_diagnostico = df_diagnostico.append({'Método': 'Redes neuronales', 'Características': RN_mejor_vector_parametros, 'Precisión': RN_mejor_prec, 'Diagnóstico': RN_predict[0]}, ignore_index=True)
df_diagnostico = df_diagnostico.append({'Método': 'KNN', 'Características': 'K='+ str(best_k), 'Precisión': KNN_prec, 'Diagnóstico': KNN_predict}, ignore_index=True)

df_diagnostico

In [ ]:
#Guardamos los dataset como csv
df_datos.to_csv("datospac_cardiopatia_py.csv", index=False)
df_diagnostico.to_csv("diagnostico_cardiopatia_py.csv", index=False)
df_nombre.to_csv("nombrepac_cardiopatia_py.csv", index=False)